In [1]:
import os
import glob
from typing import List
from dotenv import load_dotenv
from multiprocessing import Pool
from tqdm import tqdm

from langchain_community.document_loaders.base import BaseLoader
from langchain.document_loaders import (
    Docx2txtLoader,
    PyPDFLoader,
    TextLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

In [2]:
load_dotenv()

source_directory = '/docs'
chunk_size = 500
chunk_overlap = 50

LOADER_MAPPING = {
    ".docx": (Docx2txtLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".pdf": (PyPDFLoader, {}),
    # ".ppt": (UnstructuredPowerPointLoader, {'mode': 'elements', 'strategy': 'fast'}),
    # ".pptx": (UnstructuredPowerPointLoader, {'mode': 'elements', 'strategy': 'fast'}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}


def load_single_document(file_path: str) -> List[Document]:
    ext = "." + file_path.rsplit(".", 1)[-1]
    # TODO: if no extension found, parse metadata to determine file type
    if ext in LOADER_MAPPING:
        loader_class, loader_args = LOADER_MAPPING[ext]
        loader: BaseLoader = loader_class(file_path, **loader_args)
        return loader.load()

    raise ValueError(f"Unsupported file extension '{ext}'")

def load_documents(source_dir: str, ignored_files: List[str] = []) -> List[Document]:
    """
    Loads all documents from the source documents directory, ignoring specified files
    """
    files = []
    for ext in LOADER_MAPPING:
        file = glob.glob(os.path.join(source_dir, f"**/*{ext}"), recursive=True)

        while True:
            try:
                next(map(file.remove, (f for ignored_file in ignored_files for f in file if f.endswith(ignored_file))))
            except:
                break
        
        files.extend(
            file
        )

    print(files)
    with Pool(processes=os.cpu_count()) as pool:
        results = []
        with tqdm(total=len(files), desc='Loading new documents', ncols=80) as pbar:
            for i, docs in enumerate(pool.imap_unordered(load_single_document, files)):
                results.extend(docs)
                pbar.update()

    return results

def process_documents(ignored_files: List[str] = []) -> List[Document]:
    """
    Load documents and split in chunks
    """
    print(f"Loading documents from {source_directory}")

    documents = load_documents(source_directory, ignored_files)
    if not documents:
        print("No new documents to load")
        exit(0)
    print(f"Loaded {len(documents)} new documents from {source_directory}")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = text_splitter.split_documents(documents)
    print(f"Split into {len(texts)} chunks of text (max. {chunk_size} tokens each)")
    return texts



In [3]:
from models.gemini import Gemini
from embedders.gemini import GeminiEmbedder
from embedchain.config import BaseLlmConfig
from embedchain.config.embedder.google import GoogleAIEmbedderConfig
from chromadb.config import Settings
from vectordb.chroma import ChromaDB
import mosanid.prompts as prompts

import os

MODEL = "gemini-1.5-pro"
EMBEDDER = "models/text-embedding-004"

config = BaseLlmConfig()
config.model = MODEL
config.api_key = os.getenv("GOOGLE_API_KEY")
config.system_prompt = prompts.DOCS_SITE_DEFAULT_PROMPT
model = Gemini(config)

config = GoogleAIEmbedderConfig(model=EMBEDDER)
embedder = GeminiEmbedder(config)




c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
embeddings = load_documents('docs')
embeddings

['docs\\alex-rosenberg discrete mathematics5.pdf', 'docs\\relations.pdf']


Loading new documents:   0%|                              | 0/2 [00:00<?, ?it/s]

In [5]:

CHROMA_SETTINGS = Settings(
        persist_directory='db',
        anonymized_telemetry=False
)
CHROMA_SETTINGS.is_persistent = True
os.environ["EC_TELEMETRY"] = "false"

chroma = ChromaDB(embedder, CHROMA_SETTINGS)

collection = chroma._get_or_create_collection('mosanid-model')
embeddings = load_documents('docs', ['alex-rosenberg discrete mathematics5.pdf', 'relations.pptx'])
embeddings


['docs\\relations.pdf', 'docs\\relations.pptx']


Loading new documents:   0%|                              | 0/2 [00:00<?, ?it/s]

: 

: 